In [62]:
%matplotlib inline

import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import pickle

from matplotlib import pyplot as plt
from importlib import reload

from scipy.interpolate import interp1d

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


os.environ["KILOSORT2_PATH"] = '/mnt/cube/earneodo/repos/Kilosort2'
os.environ["KILOSORT3_PATH"] = '/home/finch/repos/Kilosort'
os.environ["NPY_MATLAB_PATH"] = '/mnt/cube/earneodo/repos/npy-matlab'

from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex
from ceciestunepipe.util import plotutil as pu
from ceciestunepipe.util.spike import kilosort as ks
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util.sound import spectral as sp
from ceciestunepipe.util import plotutil as pu

import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

# import spikefinder
import spikefinder
import spikefinder.spike_analysis_helper as sh
import spikefinder.filtering_helper as fh

2022-02-01 19:19:56,445 root         INFO     all modules loaded
2022-02-01 19:19:56,445 root         INFO     all modules loaded


In [63]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from math import ceil

# Load sync files and Kilosort Metadata

In [64]:
reload(et)
sess_par = {'bird': 'z_y19o20_21',
           'sess': '2021-10-27',
           'probe': 'probe_0',
           'sort': 'sort_pt'}

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

In [65]:
exp_struct

{'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21',
  'raw': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27',
  'sglx': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx',
  'kwik': '/experiment/z_y19o20_21/sglx/kwik/2021-10-27',
  'processed': '/mnt/sphere/speech_bci/processed_data/z_y19o20_21/2021-10-27/sglx',
  'derived': '/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-10-27/sglx',
  'tmp': '/experiment/tmp/tmp',
  'msort': '/experiment/tmp/z_y19o20_21/sglx/msort/2021-10-27',
  'ksort': '/experiment/tmp/z_y19o20_21/sglx/ksort/2021-10-27/sort_pt',
  'sort': '/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-10-27/sglx/sort_pt'},
 'files': {'par': '/experiment/tmp/z_y19o20_21/sglx/ksort/2021-10-27/sort_pt/params.json',
  'set': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx/settings.isf',
  'rig': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx/rig.json',
  'kwd': '/experiment/z_y19o20_21/sglx/kwik/2021-10-27/stre

In [66]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2022-02-01 19:20:12,174 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21', 'raw': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27', 'sglx': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx', 'kwik': '/experiment/z_y19o20_21/sglx/kwik/2021-10-27', 'processed': '/mnt/sphere/speech_bci/processed_data/z_y19o20_21/2021-10-27/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-10-27/sglx', 'tmp': '/experiment/tmp/tmp', 'msort': '/experiment/tmp/z_y19o20_21/sglx/msort/2021-10-27', 'ksort': '/experiment/tmp/z_y19o20_21/sglx/ksort/2021-10-27/sort_pt', 'sort': '/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-10-27/sglx/sort_pt'}, 'files': {'par': '/experiment/tmp/z_y19o20_21/sglx/ksort/2021-10-27/sort_pt/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx/rig.json', 'kwd': '/experim

['0815_unidrected_g0',
 '0900_unidrected_g0',
 '0930_directed_g0',
 '1033_undirected_g0',
 '1142_directed_g0',
 '1225_directed_g0',
 '1519_undirected_g0',
 '1550_undirected_g0',
 '1607_undirected_g0',
 '1635_directed_g0']

In [6]:
# raw_neuropix_data_g0 = '/net2/expData/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx/0815_unidrected_g0/' 
# sync_neuropix_data_g0 = '/net2/expData/speech_bci/derived_data/z_y19o20_21/2021-10-27/bout_rasters/0815_undirected_g0/' 
# sort_folder = '/net2/expData/speech_bci/derived_data/z_y19o20_21/2021-10-27/bout_rasters/0815_undirected_g0/sort_0/' 

# # [print(x[0]) for x in os.walk(sync_neuropix_data_g0)]

# os.chdir(sync_neuropix_data_g0)

In [83]:

### pick a session, epoch
reload(et)
reload(sglu)
epoch = sess_epochs[0] 
sess_par['epoch'] = epoch

exp_struct = et.sgl_struct(sess_par, epoch)
sgl_folders, sgl_files = sglu.sgl_file_struct(exp_struct['folders']['sglx'])


# os.listdir(exp_struct['folders']['derived'])

In [84]:
exp_struct

{'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/0815_unidrected_g0',
  'raw': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/0815_unidrected_g0',
  'sglx': '/mnt/sphere/speech_bci/raw_data/z_y19o20_21/2021-10-27/sglx/0815_unidrected_g0',
  'kwik': '/experiment/z_y19o20_21/sglx/kwik/2021-10-27/0815_unidrected_g0',
  'processed': '/mnt/sphere/speech_bci/processed_data/z_y19o20_21/2021-10-27/sglx/0815_unidrected_g0',
  'derived': '/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-10-27/sglx/0815_unidrected_g0',
  'tmp': '/experiment/tmp/tmp/0815_unidrected_g0',
  'msort': '/experiment/tmp/z_y19o20_21/sglx/msort/2021-10-27/0815_unidrected_g0',
  'ksort': '/experiment/tmp/z_y19o20_21/sglx/ksort/2021-10-27/0815_unidrected_g0/sort_pt',
  'sort': '/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-10-27/sglx/0815_unidrected_g0/sort_pt'},
 'files': {'par': '/experiment/tmp/z_y19o20_21/sglx/ksort/2021-10-27/0815_unidrected_g0/params.json',
  'set': '/mnt/sphere/sp

In [97]:
os.chdir(exp_struct['folders']['derived'])

# mot_dict.pkl - dictionary with motif data / metadata

**‘template’**: np.array, template used to find motifs in raw recordings <br>
**‘s_f’**: int, sample_rate of the stream used to find the template <br>
**‘s_f_nidq’**: int, sample_rate of the ‘nidq’ imec binary file (and the wav_mic.npy), computed from the syn square signal <br>
**‘s_f_ap_0’**: int, sample rate of the ‘ap0’ imec binary file, computed from the syn square signal <br>
**‘start_ms’**: list, start in ms in the time scale of the wav_mic.wav file <br>
**‘start_sample_naive’**: np.array, (for debugging: start sample expected from resampling start_ms to the nidq sample rate) <br>
**‘start_sample_nidq’**: np.array, start sample in the binary (‘nidq’) imec file (= wav_mic.npy) <br>
**‘start_t_0’**:  np.array, start in sec, relative to the binary (‘nidq’) imec file <br>
**‘start_sample_ap_0’**: np.array, start sample corresponding in the probe binary imec file (‘ap0’) <br>
**‘start_t_ap_0’**: np.array, start in sec, relative to the probe binary imec file (‘ap0’) 


In [98]:
with open('mot_dict_ap0.pkl', 'rb') as f:
    mot_dict = pkl.load(f)

mot_dict

FileNotFoundError: [Errno 2] No such file or directory: 'mot_dict_ap0.pkl'

In [99]:
print(mot_dict['start_sample_ap_0'])
print(mot_dict['start_sample_nidq'])

[11940016 11954747 11978296 12002776 33575627 33589967 33613848 40941210
 40964251 40988250]
[15919546 15939186 15970585 16003225 44766165 44785284 44817124 54586648
 54617367 54649367]


In [100]:
# template = mot_dict['template']
# plt.plot(template)

# spk_df.pkl - pandas dataframe with clusters data

**‘times’**: int, time in samples rel to the probe binary imec file (‘ap0’) <br>
**‘clusters’**: int, cluster ids <br>


In [105]:
with open('spk_df.pickle', 'rb') as f:
    spk_df = pkl.load(f)

spk_df

,times,clusters,cluster_id,nucelus,main_chan,kslabel
0,137,139,139,ra,66,good
1,140,320,320,NaN,197,good
2,189,175,175,ra,84,good
3,206,93,93,ra,52,good
4,206,177,177,ra,91,good
...,...,...,...,...,...,...
5777461,56635865,156,156,ra,75,mua
5777462,56635866,104,104,ra,58,good
5777463,56635868,222,222,ra,107,mua
5777464,56635875,139,139,ra,66,good


In [106]:
# get all the times when a spike occurs in a given cluster (time is sample in ap_0 timescale)
spk_df[spk_df['clusters']==50]['times']

52              709
114            1412
282            3063
350            3736
405            4248
             ...   
5777055    56631601
5777162    56632712
5777259    56633686
5777357    56634624
5777419    56635430
Name: times, Length: 60826, dtype: uint64

# clu_df.pkl - pandas dataframe with clusters metadata

**cluster_id**: int <br>
**KSLabel**: str ‘good’, ‘mua’ KSLabel: str ‘good’, ‘mua’ <br>
**template**: np.array, template of the spike waveform <br>
**max_chans**: np.array, channels ordered by amplitude of the waveform <br>
**main_chan**: np.array ,channel with maximum amplitude <br>
**main_7**: 7 highest amplitude channels <br>
**main_wav_7**: template in 7 highest amplitude channels <br>
**nucleus**: str / NAN, ‘hvc’, ‘ra’, NAN <br>


In [108]:
with open('clu_df.pickle', 'rb') as f:
    clu_df = pkl.load(f)

clu_df

,cluster_id,Amplitude,ContamPct,KSLabel,amp,ch,depth,fr,group,n_spikes,sh,main_chan,MSLabel,has_template,template,max_chans,main_7,main_wav_7,nucleus
3,3,2203.8,100.0,good,94.135460,5,40.0,0.671127,good,1267,0,5,good,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[5, 3, 4, 7, 6, 2, 1, 8, 9, 0, 10, 12, 14, 11,...","[1, 2, 3, 4, 5, 6, 7]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",NaN
424,511,1702.4,100.0,good,75.986816,6,60.0,0.208171,good,393,0,6,good,False,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",NaN,NaN,NaN,NaN
425,513,1736.4,100.0,good,73.471085,25,240.0,0.452362,good,854,0,25,good,False,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",NaN,NaN,NaN,ra
11,16,2657.0,100.0,good,87.184921,27,260.0,2.806340,good,5298,0,27,good,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[27, 29, 28, 25, 24, 31, 26, 23, 21, 19, 30, 2...","[24, 25, 26, 27, 28, 29, 31]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",ra
14,20,2036.0,100.0,mua,87.493256,29,280.0,1.510169,good,2851,0,29,good,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[29, 31, 33, 27, 28, 32, 26, 24, 35, 25, 30, 3...","[26, 27, 28, 29, 31, 32, 33]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",ra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,408,1101.2,100.0,mua,44.421074,260,2600.0,0.056148,NaN,106,0,260,NaN,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[260, 264, 263, 262, 258, 261, 259, 267, 265, ...","[258, 259, 260, 261, 262, 263, 264]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",NaN
364,449,2769.1,100.0,good,41.386757,306,3060.0,0.184865,NaN,349,0,306,NaN,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[306, 308, 309, 311, 304, 316, 313, 310, 312, ...","[304, 306, 308, 309, 311, 313, 316]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",NaN
366,451,816.0,100.0,good,41.838512,306,3060.0,29.731419,NaN,56129,0,306,NaN,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[306, 308, 309, 310, 311, 307, 304, 314, 313, ...","[304, 306, 307, 308, 309, 310, 311]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",NaN
372,457,3259.1,100.0,good,41.657963,316,3160.0,0.161028,NaN,304,0,316,NaN,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[316, 314, 321, 320, 310, 318, 323, 319, 311, ...","[310, 314, 316, 318, 320, 321, 323]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....",NaN


In [109]:
print(clu_df['nucleus'].unique())
print(clu_df['KSLabel'].unique())
print(clu_df['cluster_id'].unique())

[nan 'ra']
['good' 'mua']
[  3 511 513  16  20  19  22  28  23  27  32  33  35  36  37  38  52  53
  39  55  57  50  51  56  60  40  54  63  78  81 515 519  64  92 518  93
  65  68  72  99 101 102 103  94 111 116 104 106 120 105 107 108 133 136
 138 134 137 126 145 135 139 141 144 151 152 167 169 175 174 173 172 178
 176 177 188 196 200 191 202 201 208 210 203 212 218 221 225 231 232 233
 253 258 252 257 271 281 274 278 288 291 295 294 305 311 313 298 323 324
 322 320 327 330 337 333 336 343 342 349 354 351 353 362 350 363 364 358
 367 368 366 374 377 378 384 386 388 392 399 400 404 405 407 406 420   0
   1   2   4   9  10  11  13  15  18  21  26 514  62  84  87 516  79 112
 117 524 147 153 155 154 156 158 157 168 170 171 182 185 181 192 194 198
 186 187 189 204 209 214 215 211 213 217 220 222 223 228 229 224 230 234
 235 236 241 238 242 244 246 250 256 254 259 255 261 262 264 260 268 270
 266 265 272 273 277 282 284 276 285 296 287 293 300 302 303 315 297 308
 310 301 312 316 325 319 

# wav_mic.npy - Microphone waveform directly from the binary imec file 'nidq'

In [110]:
# wav_mic.npy - Microphone waveform directly from the binary imec file 'nidq'

npy_mic = np.load('wav_mic.npy')

from scipy.io import wavfile
samplerate, wav_mic = wavfile.read('wav_mic.wav')

plt.plot(wav_mic)

print(len(npy_mic))
print(len(wav_mic))


FileNotFoundError: [Errno 2] No such file or directory: 'wav_mic.npy'

In [111]:
samplerate

40000

# Load Raw Neural Neuropixel Data

In [112]:
run_meta_files = {'nidq': raw_neuropix_data_g0 + '0815_unidrected_g0_t0.nidq.meta',
 'lf_0': raw_neuropix_data_g0 + '0815_unidrected_g0_imec0/0815_unidrected_g0_t0.imec0.lf.meta',
 'ap_0': raw_neuropix_data_g0 + '0815_unidrected_g0_imec0/0815_unidrected_g0_t0.imec0.ap.meta'}

[print(k,v) for k, v in run_meta_files.items()]

NameError: name 'raw_neuropix_data_g0' is not defined

In [113]:
run_recordings = {k: sglex.SpikeGLXRecordingExtractor(sglu.get_data_meta_path(v)[0]) for k, v in run_meta_files.items()}

NameError: name 'run_meta_files' is not defined

## Get the raster matrix for GOOD clusters (units) that sit in HVC or RA

In [114]:
# clu_df.loc[clu_df['nucleus'].isnull() & (clu_df['KSLabel']=='good') | (clu_df['KSLabel']=='mua')]

In [115]:
print('RA GOOD: ', len(np.unique(clu_df.loc[((clu_df['KSLabel']=='good')) & (clu_df['nucleus'].isin(['ra'])), 'cluster_id'])))
print('RA MUA: ', len(np.unique(clu_df.loc[((clu_df['KSLabel']=='mua')) & (clu_df['nucleus'].isin(['ra'])), 'cluster_id'])))
print('HVC GOOD: ', len(np.unique(clu_df.loc[((clu_df['KSLabel']=='good')) & (clu_df['nucleus'].isin(['hvc'])), 'cluster_id'])))
print('HVC MUA: ', len(np.unique(clu_df.loc[((clu_df['KSLabel']=='mua')) & (clu_df['nucleus'].isin(['hvc'])), 'cluster_id'])))


RA GOOD:  82
RA MUA:  80
HVC GOOD:  0
HVC MUA:  0


In [40]:
#  np.unique(clu_df.loc[(clu_df['nucleus'].isnull(), 'cluster_id')])

In [31]:
# Get the raster matrix for GOOD clusters (units) that sit in HVC or RA

def get_window_spikes(spk_df, clu_list, start_sample, end_sample):
    onset = start_sample
    offset = end_sample
    
    spk_t = spk_df.loc[spk_df['times'].between(onset, offset, "neither")]
    
    spk_arr = np.zeros((clu_list.size, offset - onset))

    for i, clu_id in enumerate(clu_list):
        clu_spk_t = spk_t.loc[spk_t['clusters']==clu_id, 'times'].values
        spk_arr[i, clu_spk_t - onset] = 1
    return spk_arr
    
def get_rasters(spk_df, clu_list, start_samp_arr, span_samples):
    # returns np.array([n_clu, n_sample, n_trial])
    
    # get the window spikes for all of the clusters, for each of the start_samp_arr
    spk_arr_list = [get_window_spikes(spk_df, clu_list, x, x+span_samples) for x in start_samp_arr]
    return np.stack(spk_arr_list, axis=-1)
    
# Get 'good units' ids in HVC or RA
clu_list = np.unique(clu_df.loc[((clu_df['KSLabel']=='good') | (clu_df['KSLabel']=='mua')) & (clu_df['nucleus'].isin(['hvc'])), 
                                'cluster_id'])
print('{} neurons found'.format(len(clu_list)))

# clu_list = np.unique(clu_df.loc[((clu_df['KSLabel']=='good')),  
#                                 'cluster_id'])

# Choose the time pre & post onset of motif that you wish to observe
t_pre = -1
t_post = 4

# s_f_ap = run_recordings['ap_0']._s_f_0
fs_ap = mot_dict['s_f_ap_0']
# fs_ap = run_recordings['ap_0'].get_sampling_frequency()
t_pre_samp = int(t_pre * fs_ap)
t_post_samp = int(t_post * fs_ap)


rast_arr = get_rasters(spk_df, clu_list, mot_dict['start_sample_ap_0'] + t_pre_samp, t_post_samp - t_pre_samp) # neurons x time x epochs

46 neurons found


In [32]:
mot_dict['start_sample_ap_0']
# mot_dict['start_sample_nidq']

array([ 11678784,  11707765,  11734196,  11761016,  11841807,  15882938,
        15912188,  62300532,  62326123,  62355613,  62382674,  62429565,
        64409629,  64435009,  64464800,  64491470,  91204599,  91230610,
        91257250, 102722701, 102748350, 186772167, 186797518, 186827398,
       249852439, 249876530, 249902060, 280780251, 280809561, 289456254,
       289481275, 289516825, 296010080, 314544887, 314570027, 314638789,
       336416083, 336443083])

In [33]:
fs_ap

30000.035174165183

In [34]:
print(t_pre_samp)
print(t_post_samp)

-30000
120000


In [35]:
print(fs_ap)
print(clu_list.shape)
print(np.shape(rast_arr))

30000.035174165183
(46,)
(46, 150000, 38)


In [36]:
# Get the all of the motifs for which we have calculated the rasters:
def get_window_audio(wav_mic_npy, start_sample_list, span_samples):
    audio_arr_list = [wav_mic_npy[start_sample_list[i] : start_sample_list[i]+span_samples] for i in range(len(start_sample_list))]
    return(np.array(audio_arr_list).squeeze().tolist())

# Load continuous audio    
mic_audio = np.load('wav_mic.npy')

# fs_audio = mot_dict['s_f_nidq']
fs_audio = mot_dict['s_f']

t_pre_samp = int(t_pre * fs_audio)
t_post_samp = int(t_post * fs_audio)

audio_motifs = get_window_audio(mic_audio, mot_dict['start_sample_nidq'] + t_pre_samp, t_post_samp - t_pre_samp)

In [37]:
print(np.array(audio_motifs).shape)
print(fs_audio)
print(t_pre_samp)
print(t_post_samp)

(38, 200000)
40000
-40000
160000


### Load handlabels from TextGrid

In [39]:
# doc -> https://pypi.org/project/praatio/
from praatio import tgio

tg = tgio.openTextgrid("wav_mic2.TextGrid")
entryList = tg.tierDict["syllables"].entryList # Get all intervals
# entryList2 = tg.tierDict["syllables"].find("a") # Get the indicies of all occurrences of 'a'

# Create list of handlabels
handlabels = []

for i, interval in enumerate(entryList):
    #print(i, round(interval.start*fs_audio), round(interval.end*fs_audio), interval.label)
    # ! I had to round the fs_audio & the interval times (for number of samples in handlabels and audio array to match)
    handlabels.extend([interval.label for n in range(round(interval.end*round(fs_audio)) - round(interval.start*round(fs_audio)))])

audio_labels  = get_window_audio(handlabels, mot_dict['start_sample_nidq'] + t_pre_samp, t_post_samp - t_pre_samp)

FileNotFoundError: [Errno 2] No such file or directory: 'wav_mic2.TextGrid'

In [40]:
fs_audio

40000

In [42]:
print(len(wav_mic))
print(len(mic_audio))
print(len(mic_audio)/fs_audio)

print(len(handlabels))
print(len(handlabels)/fs_audio)

75507609
75507609
1887.690225


NameError: name 'handlabels' is not defined

In [43]:
for mot in range(len(audio_labels)):
    print(np.unique(audio_labels[mot]))

NameError: name 'audio_labels' is not defined

In [31]:
for mot in range(len(audio_labels)):
    audio_labels[mot] = [12 if x=='u' else x for x in audio_labels[mot]]            # Unlabeled period to label 12
    audio_labels[mot] = [10 if x=='I' or x=='i' else x for x in audio_labels[mot]]  # Introductory notes to label 10
    audio_labels[mot] = [7 if x=='7`' else x for x in audio_labels[mot]]            # Fix label '7''
    audio_labels[mot] = [11 if x=='C' or x=='c' else x for x in audio_labels[mot]]  # Calls to label 11
    audio_labels[mot] = [int(x) for x in audio_labels[mot]]                         # Convert to integer labels

In [32]:
for mot in range(len(audio_labels)):
    print(np.unique(audio_labels[mot]))

[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 11 12]
[ 1  2  3  4  5  7  8  9 11 12]
[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 10 11 12]
[ 1  2  3  4  5  7  8  9 11 12]


In [35]:
# import matplotlib
# import matplotlib.pyplot as plt
# import numpy as np

# # [0, 1, 2, 3, 4, 5]-Syllables.  [7]-Silence within bout.  [8]-Silence outside bout.  [9]-Intramotif Note.  [10]-Intro Note.  [11]-Call.  [12]-Unlabeled.

# mot = 8

# plt.figure()
# plt.scatter(np.linspace(0, len(audio_labels[mot]), num=len(audio_labels[mot])), audio_labels[mot])

# plt.figure()
# #           S1       S2      S3        S4        S5                               InSil     
# colors = ['brown', 'red', 'orange', 'yellow', 'green', 'blue', 'purple', 'grey', 'black', 'pink', 'cyan', 'turquoise', 'grey']

# # colors = {1:'brown', 2:'red', 3:'orange', 4:'yellow', 5:'green', 6:'blue',
# #           7:'purple', 8:'grey', 9:'black', 10:'pink', 11:'cyan', 12:'turquoise', 13:'grey'}


# fig = plt.figure(figsize=(8,8))

# plt.scatter(np.linspace(0, len(audio_labels[mot]), num=len(audio_labels[mot])), audio_motifs[mot], c=audio_labels[mot], cmap=matplotlib.colors.ListedColormap(colors))
# # plt.scatter(np.linspace(0, len(audio_labels), num=len(audio_labels)), audio_motifs[1], c=audio_labels[mot], cmap=matplotlib.colors.ListedColormap(colors))

# # cb = plt.colorbar()
# # loc = np.arange(0,max(audio_labels[mot]),max(audio_labels[mot])/float(len(colors)))
# # cb.set_ticks(loc)
# # cb.set_ticklabels(colors)

In [36]:
# def plot_motif_array(mot_list, t_0=None):
    
#     num_motifs = len(mot_list)
#     fig, ax_arr = plt.subplots(nrows=num_motifs, figsize=[5, 30], sharex=True)
    
#     for mot in range(num_motifs):
#         ax = ax_arr[mot]
#         ax.plot(mot_list[mot])        
    
#         if t_0 is not None:
#             ax.axvline(x=t_0, color='red')

# plot_motif_array(audio_motifs, t_0=-t_pre_samp)

In [37]:
# num_motifs = len(audio_motifs)

# for mot in range(num_motifs):
#     plt.plot(audio_motifs[mot])

In [38]:
# import scipy

# num_motifs = len(audio_motifs)

# for mot in range(num_motifs):
#     scipy.io.wavfile.write('motif_' + str(mot) + '.wav', round(fs_audio), np.array(audio_motifs[mot]).astype(np.int16))
    

# Prepare numpy arrays to feed into GPFA

In [38]:
# np.reshape(spikes_motifs, [2,0,1])
spikes_motifs = np.transpose(rast_arr, axes=(2, 0, 1)).tolist() 

In [39]:
print('Motifs x Neurons x time (SPIKE DOMAIN): ', np.shape(spikes_motifs), 'type: ', type(spikes_motifs))
print('motifs x time (AUDIO DOMAIN): ', np.shape(audio_motifs), 'type: ', type(audio_motifs))
if ("audio_labels" in locals()): print('motifs x time (LABELS DOMAIN): ', np.shape(audio_labels), 'type: ', type(audio_labels))

Motifs x Neurons x time (SPIKE DOMAIN):  (38, 46, 150000) type:  <class 'list'>
motifs x time (AUDIO DOMAIN):  (38, 200000) type:  <class 'list'>


## Downsample to 1ms bins

In [44]:
len(spikes_motifs)

bin_ms = 1 # miliseconds
bin_samples_audio = round(bin_ms/1000 * fs_audio)
bin_samples_spikes = round(bin_ms/1000 * fs_ap)

# BIN RASTERS
# Downsample spikes using the sum of spikes in each bins. Can be converted to spike rates
bin_spikes_motifs = []
for mot in range(len(spikes_motifs)):
    bin_spikes_motifs.append(sh.downsample_list_2d(spikes_motifs[mot], bin_samples_spikes, mode='sum')) 
    
# BIN AUDIO
# Downsample audio using the mean of the signal
bin_audio_motifs = sh.downsample_list_2d(audio_motifs, bin_samples_audio, mode='mean')

# BIN LABLES
# Downsample labels using the mode (highest repeated label) at syllable edges
if ("audio_labels" in locals()):
    bin_motif_labels = sh.downsample_list_2d(audio_labels, bin_samples_audio, mode='mode')

In [45]:
if ("audio_labels" in locals()): print(np.unique(audio_labels[0]))
if ("bin_motif_labels" in locals()): print(np.unique(bin_motif_labels[0]))

In [46]:
# cl_bn_spike_motifs = np.array(clean_spikeRaster_noisyEvents_3d(bin_spikes_motifs, coocurrence_threshold=4))
cl_bn_spike_motifs = np.array(bin_spikes_motifs)

In [47]:
print(type(cl_bn_spike_motifs), np.shape(cl_bn_spike_motifs))
print(type(bin_audio_motifs), np.shape(bin_audio_motifs))
if ("bin_motif_labels" in locals()): print(type(bin_motif_labels), np.shape(bin_motif_labels))

<class 'numpy.ndarray'> (38, 46, 5000)
<class 'numpy.ndarray'> (38, 5000)


In [63]:
# from matplotlib import cm
# magma = cm.get_cmap('magma', 12)
# cmap = magma

# # PLOT MOTIFS & RASTERS
# num_motifs2plot = 10
# spacing = 2000

# for mot in range(num_motifs2plot):
        
#     # Audio
#     audio_mot = audio_motifs[mot]
    
#     plt.figure()
#     plt.plot(audio_mot)
#     plt.xticks(np.arange(0, len(audio_mot), step=fs_audio*10), [int(i) for i in np.arange(0, len(audio_mot)/(fs_audio), step=10)])
#     plt.xlabel('Time(s)')
#     plt.ylabel('Amplitude')
#     # plt.title('Bird {}, sess {}, mot {}'.format(bird_id, session, mot))
    
#     # SPIKE RASTERS
#     spike_mot = cl_bn_spike_motifs[mot]
    
#     f, ax = plt.subplots(1, 1)
#     psm = ax.pcolormesh(spike_mot, cmap=cmap, rasterized=True)
#     f.colorbar(psm, ax=ax, orientation="horizontal")

#     ax.set_ylabel('Channel #')
#     ax.set_xlabel('Time(ms)')

# Save matrices to feed to GPFA

In [48]:
os.getcwd()

'/mnt/sphere/speech_bci/derived_data/z_y19o20_21/2021-12-23/sglx/1512_female_pin-directed_g0'

In [60]:
file_name = 'GPFA_' + sess_par['bird'] + '_' + sess_par['sess'] + '_' + sess_par['epoch'] + \
              '_38motifs_1msBin_SpikeSorted-GOOD+MUA-1s_to_4swindow-HVC.mat' 
file_name

'GPFA_z_y19o20_21_2021-12-23_1512_female_pin-directed_g0_38motifs_1msBin_SpikeSorted-GOOD+MUA-1s_to_4swindow-HVC.mat'

In [61]:

# file_name = ('GPFA_z_y19o20_21-2020_10_27-0815-10_bin_motifs_1msBin_SpikeSorted-GOOD-1s_to_4swindow-RA.mat')

import scipy.io as sio

# If there are labels:
if ("bin_motif_labels" in locals()):   
    sio.savemat(file_name, {'bin_motif_spikes':cl_bn_spike_motifs, 'bin_motif_audio':bin_audio_motifs, 'bin_motif_labels':bin_motif_labels,
                        'audio_motifs':audio_motifs, 'audio_labels':audio_labels})
else:
    sio.savemat(file_name, {'bin_motif_spikes':cl_bn_spike_motifs, 'bin_motif_audio':bin_audio_motifs,
                        'audio_motifs':audio_motifs})
